# The Nature Conservancy Fisheries Monitoring Kaggle Competition

This is an attempt to compete in the above mentioned Kaggle competition using a neural network written in Keras.

In [1]:
# Lets load us some settings!
import json


print "Loading settings..."
with open('SETTINGS.json') as settings_file:
    settings = json.load(settings_file)

# Source directory for your data
source_dir = settings['source_dir']
train_dir = settings['train_dir']
models_dir = settings['models_dir']

print "Settings loaded!"

Loading settings...
Settings loaded!


In [39]:
import numpy as np
np.random.seed(615)

import sys

sys.path.append('/usr/local/lib/python2.7/site-packages')

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.cross_validation import KFold
from keras.models import Sequential, load_model
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.optimizers import SGD, Adagrad, RMSprop, Adam, Nadam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy, categorical_crossentropy
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import log_loss
from keras.layers.advanced_activations import LeakyReLU
from keras import __version__ as keras_version
import matplotlib.pyplot as plt
%matplotlib inline


# This gets re-used for every image, so do it out of the loop
kernel_sharpen_3 = np.array([[-1,-1,-1,-1,-1],
                             [-1,2,2,2,-1],
                             [-1,2,8,2,-1],
                             [-1,2,2,2,-1],
                             [-1,-1,-1,-1,-1]]) / 8.0

def get_im_cv2(path):
    img = cv2.imread(path,1)
    
    # For color historgram
    img_yuv = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)

    # equalize the histogram of the Y channel
    img_yuv[:,:,0] = cv2.equalizeHist(img_yuv[:,:,0])

    # convert the YUV image back to RGB format
    img_output = cv2.cvtColor(img_yuv, cv2.COLOR_YUV2BGR)
    
    # Sharpen
    output_3 = cv2.filter2D(img_output, -1, kernel_sharpen_3)
    
    # Reduce to manageable size
    resized = cv2.resize(output_3, (224, 224), interpolation = cv2.INTER_LINEAR)
    return resized


def load_train():
    X_train = []
    X_train_id = []
    y_train = []
    start_time = time.time()

    print('Read train images')
    folders = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
    for fld in folders:
        index = folders.index(fld)
        print('Load folder {} (Index: {})'.format(fld, index))
        path = os.path.join(source_dir, 'fish-train', fld, '*.jpg')
        files = glob.glob(path)
        for fl in files:
            flbase = os.path.basename(fl)
            img = get_im_cv2(fl)
            X_train.append(img)
            X_train_id.append(flbase)
            y_train.append(index)

    print('Read train data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return X_train, y_train, X_train_id


def load_test():
    path = os.path.join(source_dir, 'test_stg1', 'test_stg1', '*.jpg')
    files = sorted(glob.glob(path))

    X_test = []
    X_test_id = []
    for fl in files:
        flbase = os.path.basename(fl)
        img = get_im_cv2(fl)
        X_test.append(img)
        X_test_id.append(flbase)

    return X_test, X_test_id


def create_submission(predictions, test_id):
    result1 = pd.DataFrame(predictions, columns=['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])
    result1.loc[:, 'image'] = pd.Series(test_id, index=result1.index)
    now = datetime.datetime.now()
    sub_file = 'submission_' + '_' + str(now.strftime("%Y-%m-%d-%H-%M")) + '.csv'
    result1.to_csv(sub_file, index=False)


def read_and_normalize_train_data():
    train_data, train_target, train_id = load_train()

    print('Convert to numpy...')
    train_data = np.array(train_data, dtype=np.uint8)
    train_target = np.array(train_target, dtype=np.uint8)

    print('Convert to float...')
    train_data = train_data.astype('float32')
    train_data = train_data / 255
    train_target = np_utils.to_categorical(train_target, 8)

    print('Train shape:', train_data.shape)
    print(train_data.shape[0], 'train samples')
    return train_data, train_target, train_id


def read_and_normalize_test_data():
    start_time = time.time()
    test_data, test_id = load_test()

    test_data = np.array(test_data, dtype=np.uint8)

    test_data = test_data.astype('float32')
    test_data = test_data / 255

    print('Test shape:', test_data.shape)
    print(test_data.shape[0], 'test samples')
    print('Read and process test data time: {} seconds'.format(round(time.time() - start_time, 2)))
    return test_data, test_id


def dict_to_list(d):
    ret = []
    for i in d.items():
        ret.append(i[1])
    return ret


def merge_several_folds_mean(data, nfolds):
    a = np.array(data[0])
    for i in range(1, nfolds):
        a += np.array(data[i])
    a /= nfolds
    return a.tolist()


def create_model():
    model = Sequential()

    ### Model 12
    model.add(Convolution2D(16, 3, 3, border_mode='same', 
                            W_regularizer=l2(0.01), b_regularizer=l2(0.01),
                            input_shape=(224, 224, 3)))
    model.add(LeakyReLU())

    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
    
    model.add(Dense(128, W_regularizer=l2(0.01), b_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))
    
    model.add(Dense(64, W_regularizer=l2(0.01), b_regularizer=l2(0.01)))
    model.add(LeakyReLU())
    model.add(Dropout(0.5))

    model.add(Dense(8, W_regularizer=l2(0.01), b_regularizer=l2(0.01)))
    model.add(Activation("softmax"))
    
    optimizer = RMSprop(lr=0.0001)
    
    model.compile(loss='kld', optimizer=optimizer, metrics=[categorical_crossentropy])    
    
    
    return model


def get_validation_predictions(train_data, predictions_valid):
    pv = []
    for i in range(len(train_data)):
        pv.append(predictions_valid[i])
    return pv


def run_cross_validation_create_models(nfolds=10, train_data=None, train_target=None, train_id=None):
    # input image dimensions
    batch_size = 24
    nb_epoch = 10000
    random_state = 51

    yfull_train = dict()
    kf = KFold(len(train_id), n_folds=nfolds, shuffle=True, random_state=random_state)
    num_fold = 0
    sum_score = 0
    models = []
    for train_index, test_index in kf:
        model = create_model()
        X_train = train_data[train_index]
        Y_train = train_target[train_index]
        X_valid = train_data[test_index]
        Y_valid = train_target[test_index]
        
        best_model_file = models_dir+'model_'+str(num_fold)+'.h5'
        
        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(X_train), len(Y_train))
        print('Split valid: ', len(X_valid), len(Y_valid))
        
        callbacks = [
            EarlyStopping(monitor='val_loss', patience=3, verbose=0),
            ModelCheckpoint(best_model_file, monitor='val_categorical_crossentropy', 
                            verbose = 1, save_best_only = True)
        ]
        
        model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
              shuffle=True, verbose=2, validation_data=(X_valid, Y_valid),
              callbacks=callbacks)

        predictions_valid = model.predict(X_valid.astype('float32'), batch_size=batch_size, verbose=2)
        score = log_loss(Y_valid, predictions_valid)
        print('Score log_loss: ', score)
        sum_score += score*len(test_index)

        # Store valid predictions
        for i in range(len(test_index)):
            yfull_train[test_index[i]] = predictions_valid[i]

        models.append(model)

    score = sum_score/len(train_data)
    print("Log_loss train independent avg: ", score)

    return models


def run_cross_validation_process_test(nfolds):
    batch_size = 24
    num_fold = 0
    yfull_test = []
    test_id = []
    #nfolds = len(models)
    
    # Load models from disk.
    models = []
    
    for fold in range(nfolds):
        models.append(load_model(models_dir+"model_"+str(fold)+".h5"))
    
    for i in range(nfolds):
        model = models[i]
        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        test_data, test_id = read_and_normalize_test_data()
        test_prediction = model.predict(test_data, batch_size=batch_size, verbose=2)
        yfull_test.append(test_prediction)

    test_res = merge_several_folds_mean(yfull_test, nfolds)
    create_submission(test_res, test_id)




In [20]:
X_train, y_train, X_train_id = read_and_normalize_train_data()

Read train images
Load folder ALB (Index: 0)
Load folder BET (Index: 1)
Load folder DOL (Index: 2)
Load folder LAG (Index: 3)
Load folder NoF (Index: 4)
Load folder OTHER (Index: 5)
Load folder SHARK (Index: 6)
Load folder YFT (Index: 7)
Read train data time: 138.55 seconds
Convert to numpy...
Convert to float...
('Train shape:', (3777, 224, 224, 3))
(3777, 'train samples')


In [40]:
num_folds = 5
models = run_cross_validation_create_models(num_folds, X_train, y_train, X_train_id)

Start KFold number 1 from 5
('Split train: ', 3021, 3021)
('Split valid: ', 756, 756)
Train on 3021 samples, validate on 756 samples
Epoch 1/10000
Epoch 00000: val_categorical_crossentropy improved from inf to 1.20366, saving model to /Users/thughes/tmp/data/model_0.h5
569s - loss: 4.0357 - categorical_crossentropy: 1.8800 - val_loss: 1.2037 - val_categorical_crossentropy: 1.2037
Epoch 2/10000
Epoch 00001: val_categorical_crossentropy improved from 1.20366 to 0.99046, saving model to /Users/thughes/tmp/data/model_0.h5
565s - loss: 2.7330 - categorical_crossentropy: 1.2956 - val_loss: 0.9905 - val_categorical_crossentropy: 0.9905
Epoch 3/10000
Epoch 00002: val_categorical_crossentropy improved from 0.99046 to 0.86106, saving model to /Users/thughes/tmp/data/model_0.h5
613s - loss: 2.4917 - categorical_crossentropy: 1.1126 - val_loss: 0.8611 - val_categorical_crossentropy: 0.8611
Epoch 4/10000
Epoch 00003: val_categorical_crossentropy improved from 0.86106 to 0.75404, saving model to /Us

KeyboardInterrupt: 

In [37]:
num_folds = 5
run_cross_validation_process_test(num_folds)

Start KFold number 1 from 5
('Test shape:', (1000, 224, 224, 3))
(1000, 'test samples')
Read and process test data time: 39.87 seconds
Start KFold number 2 from 5
('Test shape:', (1000, 224, 224, 3))
(1000, 'test samples')
Read and process test data time: 35.45 seconds
Start KFold number 3 from 5
('Test shape:', (1000, 224, 224, 3))
(1000, 'test samples')
Read and process test data time: 36.4 seconds
Start KFold number 4 from 5
('Test shape:', (1000, 224, 224, 3))
(1000, 'test samples')
Read and process test data time: 35.31 seconds
Start KFold number 5 from 5
('Test shape:', (1000, 224, 224, 3))
(1000, 'test samples')
Read and process test data time: 35.25 seconds
